# Inbalanced Classes
## In this lab, we are going to explore a case of imbalanced classes. 


Like we disussed in class, when we have noisy data, if we are not careful, we can end up fitting our model to the noise in the data and not the 'signal'-- the factors that actually determine the outcome. This is called overfitting, and results in good results in training, and in bad results when the model is applied to real data. Similarly, we could have a model that is too simplistic to accurately model the signal. This produces a model that doesnt work well (ever). 


### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.

### First, download the data from: https://www.kaggle.com/ntnu-testimon/paysim1. Import the dataset and provide some discriptive statistics and plots. What do you think will be the important features in determining the outcome?
### Note: don't use the entire dataset, use a sample instead, with n=100000 elements, so your computer doesn't freeze.

In [40]:
# Your code here

import pandas as pd

data = pd.read_csv("/Users/medea/Documents/Ironhack/Labs_Week_7/lab-imbalance/PS_20174392719_1491204439457_log.csv").sample(n=1000)

In [30]:
data

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
534802,21,PAYMENT,4287.87,C382961472,96.61,0.00,M965207122,0.00,0.00,0,0
26289,8,CASH_OUT,549997.27,C987348198,123140.00,0.00,C1227603646,0.00,332709.92,0,0
4331487,308,CASH_OUT,175380.36,C1702670898,71627.84,0.00,C614717878,5278009.61,5453389.96,0,0
879075,42,TRANSFER,1255097.47,C998756411,2005.00,0.00,C449911178,4801577.11,6056674.59,0,0
2179364,185,CASH_IN,148898.69,C1890644909,5868355.49,6017254.18,C498972208,1287952.04,1139053.36,0,0
...,...,...,...,...,...,...,...,...,...,...,...
431977,18,TRANSFER,1588252.33,C332106886,48965.00,0.00,C1187497097,0.00,1588252.33,0,0
2336206,189,CASH_OUT,81307.89,C11401457,0.00,0.00,C1242656943,168821.46,250129.35,0,0
5566468,393,CASH_IN,56186.19,C451176702,5176.20,61362.39,C971756929,681432.84,625246.65,0,0
5804015,401,CASH_IN,106262.47,C1467494968,5092.00,111354.47,C606759475,191496.32,0.00,0,0


### What is the distribution of the outcome? 

In [41]:
# Your response here

data["isFraud"].value_counts()

0    997
1      3
Name: isFraud, dtype: int64

### Clean the dataset. How are you going to integrate the time variable? Do you think the step (integer) coding in which it is given is appropriate?

In [32]:
# Your code here
data.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [42]:
dummies = pd.get_dummies(data["type"])
data_clean = pd.concat([data, dummies], axis=1)

data_clean.drop(columns=["type", "nameOrig","nameDest"] , inplace=True)

### Run a logisitc regression classifier and evaluate its accuracy.

In [43]:
# Your code here
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X = data_clean.drop(columns="isFraud")
y = data_clean["isFraud"]

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [44]:
data_clean["isFraud"].value_counts()

0    997
1      3
Name: isFraud, dtype: int64

In [45]:
lr = LogisticRegression()

lr.fit(X_train, y_train)

lr.score(X_test, y_test)

/Users/medea/opt/anaconda3/envs/projectsql/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.992

### Now pick a model of your choice and evaluate its accuracy.

In [46]:
# Your code here

from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train,y_train)

knn.score(X_test, y_test)

0.996

### Which model worked better and how do you know?

In [ ]:
# Your response here

# the KNN worked better

### Note: before doing the first commit, make sure you don't include the large csv file, either by adding it to .gitignore, or by deleting it.